In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import json
import os
import sys

In [2]:
course_schedule = []
NO_WINDOW = True
IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440
# SCREEN_WIDTH = 2048
# SCREEN_HEIGHT = 1152
WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

In [3]:
def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element_by_xpath(xpath)

def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [4]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')
    username.send_keys("DN20092360")
    password.send_keys("jj1216")
#     send_keys(username, "DN20092360")
#     send_keys(password, "jj1216")

    wait.until(ec.element_to_be_clickable((By.CLASS_NAME, "submit-button")))
#     wait.until(ec.presence_of_element_located((By.XPATH, "//div[@class='submit-button']/input")))

    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'BOOK THIS CLASS NOW')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False

In [5]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [6]:
temp = []

# ---------------

with open(r'./class_schedules/26 April - 02 May 2021.json', "r") as read_file:
    course_schedule = json.load(read_file)
# ---------------


c = search_courses_by_date('28', course_schedule)
c = search_courses_by_name('Hot', c)
c = search_courses_by_teacher('Ethan', c)
# c = search_courses_by_time('6', c)
temp.extend(c)

courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

Date: 28 Apr Wed
-----------------------

Name: Hot Yoga 熱瑜珈
Teacher: Ethan Lee
Time: 06:45 - 07:45 PM




In [7]:
master_counter = 1

In [8]:
def sign_in_once(driver, wait):
#     print("sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [9]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

In [10]:
csv_file_name = "./log_try_booking_a_class_that_was_full-- {}.csv".format(timestamp_string())

In [11]:
class_idx = 1
num_classes = len(courses_to_be_booked)

In [12]:
def reserve_class(driver, wait, course_to_be_booked):
    
    global master_counter
    global class_idx
    global num_classes

    stride =60 / 60 * 15 * num_classes

    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    
#     reserve_class_log_dict = {}
    
#     reserve_class_log_dict['name'] = class_name
    driver.get(book_url)
    
    if click_book_this_class_now(driver, wait):
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',BOOKED,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        num_classes = num_classes - 1
    else:
        
#         print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',FULL,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        if (master_counter - class_idx) % stride == 0:
            
            with open(csv_file_name, "a") as write_file:
                write_file.write(log_str)
        
        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=60)
        scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
        master_counter = master_counter + 1
    
    if class_idx < num_classes:
        class_idx = class_idx + 1
    else:
        class_idx = 1


In [13]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

#CSV file table header
with open(csv_file_name, "w") as write_file:
            write_file.write("#,Class,Status,Time\n")
        
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * (idx % 3), WINDOW_HEIGHT * int(idx / 3));
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)

    sign_in_once(driver, wait)
    next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=30)
    scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
    
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()

[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [/home/huohsien/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache


 
'1','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:25:53.111102'

'2','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:27:13.825530'

'3','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:28:34.529388'

'4','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:29:55.237741'

'5','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:31:19.520356'

'6','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:32:40.241744'

'7','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:34:00.923036'

'8','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:35:21.618244'

'9','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:36:42.301029'

'10','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:38:02.987120'

'11','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:39:23.669908'

'12','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:40:44.336003'

'13','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:42:05.061821'

'14','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-25 21:43:25.724807'

'15','Hot Yoga 熱瑜珈' by 'Eth

'118','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:03:23.360495'

'119','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:04:44.058796'

'120','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:06:04.748144'

'121','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:07:25.442458'

'122','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:08:46.136601'

'123','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:10:06.846822'

'124','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:11:27.515493'

'125','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:12:48.229779'

'126','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:14:08.906437'

'127','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:15:29.605586'

'128','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:16:50.290258'

'129','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:18:10.990784'

'130','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:19:31.657624'

'131','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 00:20:52.355511'

'132',

'234','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:39:43.949204'

'235','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:41:04.667461'

'236','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:42:25.384079'

'237','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:43:46.109765'

'238','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:45:06.822855'

'239','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:46:27.515097'

'240','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:47:48.198869'

'241','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:49:08.911500'

'242','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:50:29.613240'

'243','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:51:50.284014'

'244','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:53:11.000990'

'245','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:54:31.709844'

'246','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:55:52.428198'

'247','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 02:57:13.133552'

'248',

'350','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:15:45.260749'

'351','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:17:05.977019'

'352','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:18:26.639642'

'353','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:19:47.315057'

'354','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:21:07.979972'

'355','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:22:28.646113'

'356','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:23:49.346428'

'357','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:25:10.055465'

'358','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:26:30.758761'

'359','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:27:51.485004'

'360','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:29:12.189348'

'361','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:30:32.906351'

'362','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:31:53.627049'

'363','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 05:33:14.321990'

'364',

'466','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 07:51:46.142527'

'467','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 07:53:06.824151'

'468','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 07:54:27.538715'

'469','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 07:55:48.246948'

'470','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 07:57:08.957386'

'471','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 07:58:29.673190'

'472','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 07:59:50.345159'

'473','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 08:01:11.026919'

'474','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 08:02:31.731140'

'475','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 08:03:52.397122'

'476','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 08:05:13.064645'

'477','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 08:06:33.777684'

'478','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 08:07:54.500549'

'479','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 08:09:15.265995'

'480',

'582','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:27:47.421917'

'583','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:29:08.122472'

'584','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:30:28.801524'

'585','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:31:49.491900'

'586','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:33:10.185958'

'587','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:34:30.914964'

'588','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:35:51.602390'

'589','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:37:12.275616'

'590','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:38:32.955550'

'591','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:39:53.630868'

'592','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:41:14.298213'

'593','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:42:35.006166'

'594','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:43:55.694432'

'595','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 10:45:16.398962'

'596',

'698','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:03:48.989890'

'699','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:05:09.716822'

'700','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:06:30.418708'

'701','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:07:51.130506'

'702','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:09:11.848148'

'703','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:10:32.546427'

'704','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:11:53.244428'

'705','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:13:13.935098'

'706','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:14:34.612873'

'707','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:15:55.326153'

'708','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:17:16.009961'

'709','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:18:36.697038'

'710','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:19:57.401997'

'711','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 13:21:18.075403'

'712',

'814','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:39:50.419822'

'815','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:41:11.094855'

'816','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:42:31.817109'

'817','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:43:52.494781'

'818','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:45:13.192396'

'819','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:46:33.917607'

'820','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:47:54.653927'

'821','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:49:15.325501'

'822','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:50:36.039009'

'823','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:51:56.741965'

'824','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:53:17.422944'

'825','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:54:38.120440'

'826','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:55:58.818788'

'827','Hot Yoga 熱瑜珈' by 'Ethan Lee',FULL,'2021-04-26 15:57:19.499548'

'828',